**Install Required Libraries**

In [1]:
!pip install tensorflow keras matplotlib numpy gradio

**Dataset** (Download the dataset from Kaggle:[link text](https://)https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Then verify it's there:
!ls /content/drive/MyDrive/chest_xray_project/chest_xray/train

NORMAL	PNEUMONIA


**Import Libraries and Preprocess Data**

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/chest_xray_project/chest_xray/train'
val_dir   = '/content/drive/MyDrive/chest_xray_project/chest_xray/val'
test_dir  = '/content/drive/MyDrive/chest_xray_project/chest_xray/test'

train_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(train_dir, target_size=(150,150), batch_size=32, class_mode='binary')
val_data   = val_gen.flow_from_directory(val_dir, target_size=(150,150), batch_size=32, class_mode='binary')
test_data  = test_gen.flow_from_directory(test_dir, target_size=(150,150), batch_size=32, class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


**Build the CNN Model**

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

**Train the Model**

In [9]:
# Now train your CNN model on the dataset:
history = model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10
 52/163 ━━━━━━━━━━━━━━━━━━━━ 10:32 6s/step - accuracy: 0.8633 - loss: 0.3321

KeyboardInterrupt: 

**Evaluate the Model**

In [10]:
# After training completes,test the model accuracy:
loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc*100:.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 148s 7s/step - accuracy: 0.8152 - loss: 0.4913
Test Accuracy: 79.97%


**Save the Model**

In [11]:
model.save('/content/pneumonia_cnn_model.h5')

In [13]:
!ls /content

drive  pneumonia_cnn_model.h5  sample_data


In [14]:
from google.colab import files
files.download('/content/pneumonia_cnn_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Create the User Interface (Prediction App)**

In [12]:
# This lets you upload any X-ray image and get a prediction (Normal or Pneumonia).

!pip install gradio
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('/content/pneumonia_cnn_model.h5')

def predict_pneumonia(img):
    img = img.resize((150,150))
    img_array = np.array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    return "🩺 Pneumonia Detected" if prediction > 0.5 else "✅ Normal Lungs"

app = gr.Interface(
    fn=predict_pneumonia,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Chest X-Ray Pneumonia Detector",
    description="Upload a chest X-ray image to check for pneumonia."
)

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a7007a9061e240a9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
